<a href="https://colab.research.google.com/github/KevinWahle/ASSD-TPF/blob/master/ASSD_Dialization%2BTranscriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dializator + Transcriber

El objetivo de este notebook es: ...


## Descarga e importación de librerías


In [ ]:
import os
os.system("pip install git+https://github.com/openai/whisper.git")
os.system("pip install git+https://github.com/resemble-ai/Resemblyzer.git")
os.system("pip install pathlib")
os.system("pip install librosa==0.9.2")

import whisper
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from scipy.io import wavfile
import numpy as np

## Dialización

### Carga de Oradores

In [ ]:
wav_fpath = "/content/sample.wav"
wav = preprocess_wav(wav_fpath)
sampling_rate=16000

segments = [[0, 5.5], [6.5, 12], [17, 25]]                  #TODO: Ver cómo integrar a la GUI
speaker_names = ["Kyle Gass", "Sean Evans", "Jack Black"]
speaker_wavs = [wav[int(s[0] * sampling_rate):int(s[1] * sampling_rate)] for s in segments]

### Aplicación de la Red Neuronal

In [ ]:
## Compare speaker embeds to the continuous embedding of the interview
# Derive a continuous embedding of the interview. We put a rate of 16, meaning that an 
# embedding is generated every 0.0625 seconds. It is good to have a higher rate for speaker 
# diarization, but it is not so useful for when you only need a summary embedding of the 
# entire utterance. A rate of 2 would have been enough, but 16 is nice for the sake of the 
# demonstration. 
# We'll exceptionally force to run this on CPU, because it uses a lot of RAM and most GPUs 
# won't have enough. There's a speed drawback, but it remains reasonable.
encoder = VoiceEncoder("cpu")
print("Running the continuous embedding on cpu, this might take a while...")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16) 

# Get the continuous similarity for every speaker. It amounts to a dot product between the 
# embedding of the speaker and the continuous embedding of the interview
speaker_embeds = [encoder.embed_utterance(speaker_wav) for speaker_wav in speaker_wavs]
similarity_dict = {name: cont_embeds @ speaker_embed for name, speaker_embed in 
                   zip(speaker_names, speaker_embeds)}

Loaded the voice encoder model on cpu in 0.01 seconds.
Running the continuous embedding on cpu, this might take a while...


### Prefiltrado de la dialización
En esta etapa se busca: ... 


In [ ]:
talk_names =[]
MIN_PROB = 65/100


for i in range(len(similarity_dict[speaker_names[0]])):     # Create an array with speaker predominance in each sample
    prev = 0
    for names in speaker_names:
        prev= max(similarity_dict[names][i], prev)
        if(prev==similarity_dict[names][i]): name = names
    if prev<MIN_PROB: name = None
    talk_names.append(name)

# Si tuve 1 valor de 5 diferente, corregilo
for i in range(len(talk_names)):                            # Soft change matrix 
    if 1<i<len(talk_names)-3 and talk_names[i-1]==talk_names[i-2]==talk_names[i+1]==talk_names[i+2]:
        talk_names[i]=talk_names[i-1]

# Crea arreglo con tiempos y nombres [[start,end,speaker]]
name=talk_names[0]
speakers_time = []
prev_name=0
io=0
for i in range(len(talk_names)):
    name=talk_names[i]
    if name!=prev_name:
        if prev_name!=None:
            speakers_time.append([io,i,prev_name,""])
        io=i+1
    prev_name=name

# Si el delta_tiempo es menor a MIN_TIME, sacamelo
# Si son tempos cercanos (menor a MIN_TIME), hace merge
MIN_TIME = 4
time_processed=[]
for i in range(len(speakers_time)):
    #Do merge
    if len(time_processed) and speakers_time[i][2]==time_processed[-1:][0][2] and (speakers_time[i][0]-time_processed[-1:][0][1])<=MIN_TIME:
        temp = time_processed[-1:][0][0]            # Guardo de la iteración anterior 
        time_processed.pop()                        # Saco el tiempo viejo
        time_processed.append([temp, speakers_time[i][1], speakers_time[i][2], ""]) # Agrego el merge
    #Delete time
    elif (speakers_time[i][1]-speakers_time[i][0])>MIN_TIME:
        if len(time_processed) and time_processed[-1:][0][1]-time_processed[-1:][0][0]<=MIN_TIME:
            time_processed.pop()                    # Si el de la iteración anteior era menor, lo saco
        time_processed.append(speakers_time[i])     # Agrego el tiempo si es mayor a mintime
    #Append and check in next iteration
    elif i<len(speakers_time)-1:
        if len(time_processed) and time_processed[-1:][0][1]-time_processed[-1:][0][0]<=MIN_TIME:
            time_processed.pop()                    # Si el de la iteración anteior era menor, lo saco
        time_processed.append(speakers_time[i])

### Visualizacion

In [ ]:
## Run the interactive demo
#interactive_diarization(similarity_dict, wav, wav_splits)

## Speech to Text

### Creación de wavs por orador

In [19]:
# Creacion de los wavs ya separados en los distintos tramos por orador
def wavCreation(slices):
    if not os.path.exists("path/to/demo_folder"):
      os.makedirs("/content/temp")

    for slice_index in range(len(slices)):
        sliceStart=slices[slice_index][0]
        sliceEnd=slices[slice_index][1]
        wavfile.write("/content/temp/slice"+str(slice_index)+".wav", sampling_rate, wav[sliceStart:sliceEnd].astype(np.float32))

wavCreation(time_processed)

### Transcripción

In [ ]:
def speech2Text(slices):
    model = whisper.load_model("base")          #TODO: Revisar tamaño modelo
    for slice_index in range(len(slices)):
        result = model.transcribe("/content/temp/slice"+str(slice_index)+".wav")
        os.remove("/content/temp/slice"+str(slice_index)+".wav")
        slices[slice_index][3]=result['text']

speech2Text(time_processed)

/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/pytho

## Resultado final


In [ ]:
print(time_processed)

[[1, 92, 'Kyle Gass', ''], [95, 195, 'Sean Evans', ''], [202, 207, 'Kyle Gass', ''], [213, 535, 'Jack Black', ' you'], [538, 564, 'Kyle Gass', ''], [585, 594, 'Kyle Gass', ''], [595, 628, 'Jack Black', ''], [629, 744, 'Sean Evans', ''], [745, 791, 'Kyle Gass', ''], [792, 825, 'Jack Black', ''], [826, 908, 'Kyle Gass', ''], [928, 933, 'Jack Black', ''], [934, 1184, 'Sean Evans', ' you'], [1185, 1221, 'Jack Black', ''], [1228, 1255, 'Kyle Gass', ''], [1265, 1390, 'Jack Black', ''], [1391, 1408, 'Kyle Gass', ''], [1467, 1764, 'Jack Black', ' you'], [1765, 1790, 'Kyle Gass', ''], [1791, 1847, 'Jack Black', ''], [1848, 1892, 'Kyle Gass', ''], [1893, 1997, 'Jack Black', ''], [2004, 2028, 'Kyle Gass', ''], [2029, 2051, 'Jack Black', '']]


#TODO
* Agregar opcion debugger
* Integrar UI